# 基于ResNeXt50的飞禽分类模型

## 一、ResNeXt网络模型

### 历史

 神经网络普遍存在的一个问题，如果要提高模型的准确率，往往采取加深网络或者加宽网络的方法。虽然这种方法是有效的，但是随之而来的，是网络设计的难度和计算开销的增加。为了一点精度的提升往往需要付出更大的代价。因此，需要一个更好的策略，在不额外增加计算代价的情况下，提升网络的精度。由此，何凯明团队在2017年CVPR会议上提出ResNeXt新型图像分类网络，引入cardinality的概念，通过控制相同的拓扑结构。

### 模型原理

传统的 split-transform-merge 结构，具有不同分支的不同拓扑结构的特征，调整 Inception 的内部结构对应着大量的超参数，这些超参数调整起来是非常困难。所以作者的思想是每个结构使用相同的拓扑结构，那么这时候的  Inception（这里简称简化 Inception) 表示为
$$
\mathcal{F}=\sum_{i=1}^{C} \mathcal{T}_{i}(\mathbf{x})
$$

下图是ResNet（左）与ResNeXt（右）block的差异。在ResNet中，输入的具有256个通道的特征经过1×1卷积压缩4倍到64个通道，之后3×3的卷积核用于处理特征，经1×1卷积扩大通道数与原特征残差连接后输出。ResNeXt也是相同的处理策略，但在ResNeXt中，输入的具有256个通道的特征被分为32个组，每组被压缩64倍到4个通道后进行处理。32个组相加后与原特征残差连接后输出。这里cardinatity指的是一个block中所具有的相同分支的数目。

![](./img/fig1.png)

下图是InceptionNet的两种 inception module结构，左边是inception module的naive版本，右边是使用了降维方法的inception module。相较于右边，左边很明显的缺点就是参数大，计算量巨大。使用不同大小的卷积核目的是为了提取不同尺度的特征信息，对于图像而言，多尺度的信息有助于网络更好地对图像信息进行选择，并且使得网络对于不同尺寸的图像输入有更好的适应能力，但多尺度带来的问题就是计算量的增加。因此在右边的模型中，InceptionNet很好地解决了这个问题，首先是1×1的卷积用于特征降维，减小特征的通道数后再采取多尺度的结构提取特征信息，在降低参数量的同时捕获到多尺度的特征信息。

![image-20220822160010450](./img/fig9.jpg)

ResNeXt正是借鉴了这种“分割-变换-聚合”的策略，但用相同的拓扑结构组建ResNeXt模块。每个结构都是相同的卷积核，保持了结构的简洁，使得模型在编程上更方便更容易。



### 网络结构

如下图，左边是ResNet的基本结构，右边是ResNeXt的基本结构：

![fig3](img/fig3.png)



residual connection就是公式中的x直接连过来，然后剩下的是32组独立的同样结构的变换，最后再进行融合，符合split-transform-merge的模式。

split-transform-merge是通用的神经网络的标准范式，基本的神经元符合这个范式。而如下图所示：

![img](img/fig4.png)

a是ResNeXt基本单元，如果把输出那里的1x1合并到一起，得到等价网络b拥有和Inception-ResNet相似的结构，而进一步把输入的1x1也合并到一起，得到等价网络c则和通道分组卷积的网络有相似的结构。

事实上，该模型说明了Inception-ResNet和通道分组卷积网络，都只是ResNeXt这一范式的特殊形式而已，进一步说明了split-transform-merge的普遍性和有效性，以及抽象程度更高，更本质一点。

下面来看ResNeXt具体的网络结构。

类似ResNet，作者选择了很简单的基本结构，每一组C个不同的分支都进行相同的简单变换，下面是ResNeXt-50（32x4d）的配置清单，32指进入网络的第一个ResNeXt基本结构的分组数量C（即基数）为32，4d表示depth即每一个分组的通道数为4（所以第一个基本结构输入通道数为128）：

![img](img/fig5.png)

可以看到ResNet-50和ResNeXt-50（32x4d）拥有相同的参数，但是精度却更高。

具体实现上，因为1x1卷积可以合并，就合并了，代码更简单，并且效率更高。

参数量不变，但是效果太好，这个时候通常会有一个『但是』。。。但是，因为分组了，多个分支单独进行处理，所以相交于原来整个一起卷积，硬件执行效率上会低一点，训练ResNeXt-101（32x4d）每个mini-batch要0.95s，而ResNet-101只要0.70s，虽然本质上计算量是相同的，通过底层的优化因为能缩小这个差距。



## 二、数据集及预处理

数据集CUB_200_2011是由加州理工学院2011年提供，是用于细粒度视觉分类任务的最广泛使用的数据集。它包含属于鸟类的 200 个子类别的 11,788 张图像。每个图像都有详细的注释：1 个子类别标签、15 个部分位置、312 个二进制属性和 1 个边界框。
从数据集网站得到对应链接https://data.caltech.edu/tindfiles/serve/1239ea37-e132-42ee-8c09-c383bb54e7ff/，但因为这一链接是跳转到真正下载链接的，故使用`curl`命令得到真正的下载链接https://tind-caltechdata.s3.amazonaws.com:443/fba51a3dc5cf42efad2c64225172351d?filename="CUB_200_2011.tgz%22"，使用`wget`下载得到数据集。

如下是CUB_200_2011数据集原始的树状目录。
```bash
.
├── attributes
│   ├── certainties.txt
│   ├── class_attribute_labels_continuous.txt
│   └── image_attribute_labels.txt
├── bounding_boxes.txt
├── classes.txt
├── image_class_labels.txt
├── images
│   ├── 001.Black_footed_Albatross
│   ├── 002.Laysan_Albatross
│   ...
│   └── 200.Common_Yellowthroat
├── images.txt
├── parts
│   ├── part_click_locs.txt
│   ├── part_locs.txt
│   └── parts.txt
├── README
└── train_test_split.txt
```

为对该数据集分类，编写如下的`./src/split.py`。

In [ ]:
import os
import pandas as pd
from PIL import Image
from shutil import copyfile

def makedir(path):
    '''
    if path does not exist in the file system, create it
    '''
    
    if not os.path.exists(path):
        os.makedirs(path)

# set paths
rootpath = './CUB_200_2011/'
imgspath = rootpath + 'images/'
trainpath = 'datasets/cub200_cropped/train_cropped/'
testpath = 'datasets/cub200_cropped/test_cropped/'

# read img names, bounding_boxes
names = pd.read_table(rootpath + 'images.txt', delimiter=' ', names=['id', 'name'])
names = names.to_numpy()
boxs = pd.read_table(rootpath + 'bounding_boxes.txt', delimiter=' ',
                     names=['id', 'x', 'y', 'width', 'height'])
boxs = boxs.to_numpy()

# crop imgs
for i in range(11788):
    im = Image.open(imgspath + names[i][1])
    im = im.crop((boxs[i][1], boxs[i][2], boxs[i][1] + boxs[i][3], boxs[i][2] + boxs[i][4]))
    im.save(imgspath + names[i][1], quality=95)
    print('{} imgs cropped and saved.'.format(i + 1))
print('All Done.')

# mkdir for cropped imgs
folders = pd.read_table(rootpath + 'classes.txt', delimiter=' ', names=['id', 'folder'])
folders = folders.to_numpy()
for i in range(200):
    makedir(trainpath + folders[i][1])
    makedir(testpath + folders[i][1])

# split imgs
labels = pd.read_table(rootpath + 'train_test_split.txt', delimiter=' ', names=['id', 'label'])
labels = labels.to_numpy()
for i in range(11788):
    if(labels[i][1] == 1):
        copyfile(imgspath + names[i][1], trainpath + names[i][1])
    else:
        copyfile(imgspath + names[i][1], testpath + names[i][1])
    print('{} imgs splited.'.format(i + 1))
print('All Done.')


在和`CUB_200_2011/`所在的文件夹路径下运行`split.py`，完成数据集分类，得到如下的树状路径。
```bash
.
└── datasets
    ├── test
    └── train
```

## 三、实践

1. 使用`git`下载下`models`库下的模型，测试发现出现错误。查看报错log发现错误出现在如下部分。

In [ ]:
'''src/dataset.py'''
import os
from PIL import Image, ImageFile
from mindspore.common import dtype as mstype
import mindspore.dataset as de
import mindspore.dataset.transforms as C
import mindspore.dataset.vision as V_C
from src.utils.sampler import DistributedSampler

...

transform_img = [
                V_C.RandomCropDecodeResize(image_size, scale=(0.08, 1.0), ratio=(0.75, 1.333)),
                V_C.RandomHorizontalFlip(prob=0.5),
                V_C.RandomColorAdjust(brightness=0.4, contrast=0.4, saturation=0.4),
                V_C.Normalize(mean=mean, std=std),
                V_C.HWC2CHW()
            ]

查看`mindspore`官方文档发现如上部分中的`RandomCropDecodeResize`等一系列函数均位于`mindspore.dataset.vision.c_transforms`和`mindspore.dataset.transforms.c_transforms`下，故对原本代码进行了添加修改。

2. 对`default_config.yaml`文件做修改，修改条目如下：
```yaml
data_path: "/data1/CUB_200_2011/datasets/train"
output_path: "./output/mytrain"
image_size: [224,224]
num_classes: 200
```
发现因为服务器配置不足，无法运行，调小`image_size`为[30,30]后使用`eval.py`评估结果，发现结果如下

![](img/fig10.png)

准确度过低，判断可能是`image_size`过小的原因，调大`image_size`为[200,200]后调整参数如下再次测试，发现结果大大增强。
```yaml
```


![](img/fig11.jpg)

3. 尝试部署到手机端。

![](img/fig12.jpg)

结果过差，分析是数据集较小的原因。

## 四、难点

1. 配环境
因为之前没有使用过ssh远程服务器，所以连接远程服务器，在远程服务器上配置环境均对我们小组的实验产生了不同程度的困扰。同时，远程服务器上的`/data0/get_start.txt`文件最初并不够完善。

2. 算力资源问题
由于本次实验是十四个小组共用同一台服务器，所以很多时候运行程序都会遇上算力不足无法运行的情况，只有在较多人不在使用算力时才可以运行成功，对我们的调试造成了一些困难。

## 五、总结

本次实验我们了解了ResNeXt50网络模型的原理，并着手使用MindSpore框架实现该网络，利用下载的CUB_200_2011数据集进行训练，评估结果较为可观，但部署到端上再测试却出现了一些结果的问题，总的来说收获不小，但也有缺陷。